# 2. Preprocessing

In [1]:
import pandas as pd
import numpy as np
from src.diametrics import transform, preprocessing
import copy

In [2]:
dexcom_data = transform.transform_directory(directory='tests/test_data/dexcom/', device='dexcom')
libre1 = transform.open_file('tests/test_data/libre/libre_amer_01.csv')
libre1_transformed = transform.convert_libre(libre1)
dxcm2 = transform.open_file('tests/test_data/dexcom/dexcom_eur_02.xlsx')
dxcm2_transformed = transform.convert_dexcom(dxcm2)

/home/cat/Desktop/projects/diametrics/src/diametrics/transform.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['time'] = pd.to_datetime(df['time'])


## 2.1. Check if df is usable

In [17]:
help(preprocessing.check_df)

Help on function check_df in module src.diametrics.preprocessing:

check_df(df)
    Check if the given object is a valid DataFrame.
    
    Args:
        df (object): The object to be checked.
    
    Returns:
        bool: True if the object is a valid DataFrame, False otherwise.
    
    Note:
        - If the object is not a DataFrame, a warning will be issued.
        - If the DataFrame has null values in the 'time' or 'glc' columns, it will be considered invalid.
        - An empty DataFrame will also be considered invalid.



In [3]:
preprocessing.check_df(dexcom_data)


True

In [4]:
preprocessing.check_df(dxcm2_transformed)


True

## 2.2. Replacing the lo/hi cutoff values

In [18]:
help(preprocessing.replace_cutoffs)

Help on function replace_cutoffs in module src.diametrics.preprocessing:

replace_cutoffs(df, remove=False, cap=True, lo_cutoff=2.1, hi_cutoff=22.3)
    Replace values in the 'glc' column of the given DataFrame based on specified cutoffs.
    
    Args:
        df (pandas.DataFrame): The DataFrame in which values will be replaced.
        remove (bool, optional): Indicates whether to remove rows with non-numeric 'glc' values. Defaults to False.
        cap (bool, optional): Indicates whether to cap values above the high cutoff and below the low cutoff. Defaults to True.
        lo_cutoff (float, optional): The low cutoff value for replacing 'glc' values. Defaults to 2.1.
        hi_cutoff (float, optional): The high cutoff value for replacing 'glc' values. Defaults to 22.3.
    
    Returns:
        pandas.DataFrame: The modified DataFrame with replaced 'glc' values and optionally removed rows.
    
    Note:
        - If remove is True, the function replaces non-numeric values in the 

In [5]:
dexcom_data = preprocessing.replace_cutoffs(dexcom_data)

In [6]:
libre1_transformed = preprocessing.replace_cutoffs(libre1_transformed, lo_cutoff=2.1, hi_cutoff=27.8)

In [7]:
dxcm2_transformed = preprocessing.replace_cutoffs(dxcm2_transformed, remove=True)

## 2.3. Change start and end times

### 2.3.1. Individual with one start time for all

In [19]:
help(preprocessing.set_time_frame)

Help on function set_time_frame in module src.diametrics.preprocessing:

set_time_frame(df, window)
    Filter the given DataFrame based on the specified time period.
    
    Args:
        df (pandas.DataFrame): The DataFrame to be filtered.
        period (list or dict): The time period to filter the DataFrame. It can be specified as a list or a dictionary.
    
            - If period is a list, it should contain two elements representing the start and end times of the period.
            Only the rows with 'time' values greater than or equal to the start time and less than the end time will be returned.
    
            - If period is a dictionary, it should contain key-value pairs where the keys represent IDs and the values
            are two-element lists representing the start and end times for each ID. The function will return all rows
            where the 'ID' column matches the specified IDs and the 'time' values fall within the respective time windows.
    
    Returns:
  

In [8]:
preprocessing.set_time_frame(libre1_transformed, ['2021-03-21', '2021-04-01'])

,time,glc,scan_glc
26,2021-03-21 00:08:00,188,NaN
27,2021-03-21 00:23:00,178,NaN
28,2021-03-21 00:38:00,163,NaN
29,2021-03-21 00:53:00,160,NaN
30,2021-03-21 01:08:00,158,NaN
...,...,...,...
1077,2021-03-31 22:53:00,162,NaN
1078,2021-03-31 23:08:00,156,NaN
1079,2021-03-31 23:23:00,155,NaN
1080,2021-03-31 23:38:00,145,NaN


### 2.3.2. Using a dictionary

In [9]:
d = {'dexcom_eur_01': ['2023-03-13 03:00:00', '2023-03-20 14:30:00'],
     'dexcom_eur_02': ['2023-03-09', '2023-03-18'],
     'dexcom_eur_03': ['2023-03-14', '2023-03-21']}

In [10]:
preprocessing.set_time_frame(dexcom_data, d)

,time,glc,ID
0,2023-03-13 03:04:11,14.7,dexcom_eur_01
1,2023-03-13 03:09:11,14.7,dexcom_eur_01
2,2023-03-13 03:14:11,14.2,dexcom_eur_01
3,2023-03-13 03:19:11,13.8,dexcom_eur_01
4,2023-03-13 03:24:11,13.8,dexcom_eur_01
...,...,...,...
6562,2023-03-20 23:35:18,7.2,dexcom_eur_03
6563,2023-03-20 23:40:18,7.3,dexcom_eur_03
6564,2023-03-20 23:45:18,7.3,dexcom_eur_03
6565,2023-03-20 23:50:18,7.3,dexcom_eur_03


## 2.4. Interpolate missing data

In [20]:
help(preprocessing.fill_missing_data)

Help on function fill_missing_data in module src.diametrics.preprocessing:

fill_missing_data(df, interval=5, method='pchip', limit=30, order=5)
    Interpolate missing data in a time series using a specified interpolation method.
    
    Args:
        df (pandas.DataFrame): The time series dataset with a 'time' column and a numeric data column.
        interval (int): The resampling interval in minutes. Defaults to 5.
        method (str): The interpolation method to use. Defaults to 'pchip'.
        limit (int): The maximum number of consecutive missing data points to interpolate. Defaults to 30.
        order (int): The order of the interpolation method if 'method' is 'polynomial' or 'spline'. Defaults to 5.
    
    Returns:
        pandas.DataFrame: The DataFrame with missing data interpolated using the specified method.
    
    Note:
        - The 'time' column should be set as the DataFrame index before calling this function.
        - The 'time' column should be in datetime f

In [11]:
# Create a chunk of data from the transformed DataFrame
dxcm2_chunk = copy.copy(dxcm2_transformed.head(30))
# Set certain rows in the 'glc' column to NaN to simulate missing data
dxcm2_chunk.loc[[4,5,14,15,16,17,18,19,20,21,26,27],'glc'] = np.nan
# Display the first 10 rows of the chunk DataFrame
dxcm2_chunk.head(10)

,time,glc
0,2023-03-08 00:00:44,10.4
1,2023-03-08 00:05:44,10.3
2,2023-03-08 00:10:44,10.2
3,2023-03-08 00:15:44,10.1
4,2023-03-08 00:20:44,NaN
5,2023-03-08 00:25:44,NaN
6,2023-03-08 00:30:44,9.9
7,2023-03-08 00:35:44,9.9
8,2023-03-08 00:40:44,9.5
9,2023-03-08 00:45:44,9.5


In [12]:
# Fill missing data using the preprocessing function
preprocessing.fill_missing_data(dxcm2_chunk, interval=5, method='pchip', limit=30).head(10)

,time,glc
0,2023-03-08 00:00:00,10.4
1,2023-03-08 00:05:00,10.3
2,2023-03-08 00:10:00,10.2
3,2023-03-08 00:15:00,10.1
4,2023-03-08 00:20:00,10.0
5,2023-03-08 00:25:00,9.9
6,2023-03-08 00:30:00,9.9
7,2023-03-08 00:35:00,9.9
8,2023-03-08 00:40:00,9.5
9,2023-03-08 00:45:00,9.5


In [13]:
libre1_chunk = copy.copy(libre1_transformed.head(30))
libre1_chunk.loc[[4,5,14,15,16,17,18,19,20,21,26,27],'glc'] = np.nan
libre1_chunk.head(10)

,time,glc,scan_glc
0,2021-03-20 17:38:00,127.0,NaN
1,2021-03-20 17:53:00,124.0,NaN
2,2021-03-20 18:08:00,121.0,NaN
3,2021-03-20 18:23:00,131.0,NaN
4,2021-03-20 18:38:00,NaN,NaN
5,2021-03-20 18:53:00,NaN,NaN
6,2021-03-20 19:08:00,166.0,NaN
7,2021-03-20 19:23:00,165.0,NaN
8,2021-03-20 19:38:00,162.0,NaN
9,2021-03-20 19:53:00,154.0,NaN


In [14]:
preprocessing.fill_missing_data(libre1_chunk, interval=15, method='linear', limit=45).head(10)

,time,glc,scan_glc
0,2021-03-20 17:30:00,127.0,NaN
1,2021-03-20 17:45:00,124.0,NaN
2,2021-03-20 18:00:00,121.0,NaN
3,2021-03-20 18:15:00,131.0,NaN
4,2021-03-20 18:30:00,142.7,NaN
5,2021-03-20 18:45:00,154.3,NaN
6,2021-03-20 19:00:00,166.0,NaN
7,2021-03-20 19:15:00,165.0,NaN
8,2021-03-20 19:30:00,162.0,NaN
9,2021-03-20 19:45:00,154.0,NaN


## 2.5. Change units

In [21]:
help(preprocessing.change_units)

Help on function change_units in module src.diametrics.preprocessing:

change_units(df)
    Convert glucose units in the DataFrame to a different unit based on a condition.
    
    Args:
        df (pandas.DataFrame): The DataFrame containing a 'glc' column with glucose readings.
    
    Returns:
        pandas.DataFrame: The DataFrame with glucose units converted based on the condition.
    
    Note:
        - The function checks the minimum glucose value in the DataFrame.
        - If the minimum value is greater than 50, the glucose units are converted to a different unit by multiplying with 0.0557 and rounding to one decimal place.
        - If the minimum value is less than or equal to 50, the glucose units are converted by multiplying with 0.0557 and rounding to the nearest integer.



In [15]:
preprocessing.change_units(libre1_transformed)

,time,glc,scan_glc
0,2021-03-20 17:38:00,7.1,NaN
1,2021-03-20 17:53:00,6.9,NaN
2,2021-03-20 18:08:00,6.7,NaN
3,2021-03-20 18:23:00,7.3,NaN
4,2021-03-20 18:38:00,8.5,NaN
...,...,...,...
1334,2021-04-03 15:08:00,7.0,NaN
1335,2021-04-03 15:23:00,6.6,NaN
1336,2021-04-03 15:38:00,6.1,NaN
1337,2021-04-03 15:53:00,6.0,NaN


In [16]:
preprocessing.change_units(dxcm2_transformed)

,time,glc
0,2023-03-08 00:00:44,187.0
1,2023-03-08 00:05:44,185.0
2,2023-03-08 00:10:44,183.0
3,2023-03-08 00:15:44,181.0
4,2023-03-08 00:20:44,178.0
...,...,...
3890,2023-03-21 15:10:57,117.0
3891,2023-03-21 15:15:57,122.0
3892,2023-03-21 15:20:57,124.0
3893,2023-03-21 15:25:57,129.0
